# Transforming plant utilization data

Combines generation and capacity data for California plants for each year dating back to 2001. Generates a utilization percentage for each plant by combining values from the two data sets.

In [1]:
import os
import settings
import pandas as pd
from simpledbf import Dbf5

SQLalchemy is not installed. No support for SQL output.


In [2]:
pd.options.mode.chained_assignment = None

## Generation

List all the files to load with their pandas configuration

In [3]:
gen_file_list = [
    dict(path="f906920y2001.xls", skiprows=7, year=2001, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path="f906920y2002.xls", skiprows=7, year=2002, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path="f906920_2003.xls", skiprows=7, year=2003, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path="f906920_2004.xls", skiprows=7, year=2004, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path="f906920_2005.xls", skiprows=7, year=2005, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path="f906920_2006.xls", skiprows=7, year=2006, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path="f906920_2007.xls", skiprows=7, year=2007, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path="eia923December2008.xls", skiprows=7, year=2008, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path='EIA923 SCHEDULES 2_3_4_5 M Final 2009 REVISED 05252011.XLS', skiprows=7, year=2009, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path='EIA923 SCHEDULES 2_3_4_5 Final 2010.xls', skiprows=7, year=2010, columns=["Plant ID", "Plant Name", "Operator Name", "Operator ID", "NET GENERATION (megawatthours)"]),
    dict(path='EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx', skiprows=5, year=2011, columns=["Plant Id", "Plant Name", "Operator Name", "Operator Id", 'Net Generation (Megawatthours)']),
    dict(path='EIA923_Schedules_2_3_4_5_M_12_2012_Final_Revision.xlsx', skiprows=5, year=2012, columns=["Plant Id", "Plant Name", "Operator Name", "Operator Id", 'Net Generation\n(Megawatthours)']),
    dict(path='EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx', skiprows=5, year=2013, columns=["Plant Id", "Plant Name", "Operator Name", "Operator Id", 'Net Generation (Megawatthours)']),
    dict(path='EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx', skiprows=5, year=2014, columns=["Plant Id", "Plant Name", "Operator Name", "Operator Id", 'Net Generation\n(Megawatthours)']),
    dict(path='EIA923_Schedules_2_3_4_5_M_12_2015_Final_Revision.xlsx', skiprows=5, year=2015, columns=["Plant Id", "Plant Name", "Operator Name", "Operator Id", 'Net Generation\n(Megawatthours)']),
]

Open all of the files and join them

In [20]:
gen_columns = [
    'plant_id',
    'plant_name',
    'operator_name',
    'operator_id',
    'net_generation_mwh',
    'year',
]

In [21]:
def standardize_generation(**kwargs):
    """
    Standardizes the given generation file to a common set of fields.
    
    Returns a DataFrame.
    """   
    # Read in the file
    df = pd.read_excel(
        os.path.join(settings.input_dir, kwargs['path']),
        sheetname='Page 1 Generation and Fuel Data',
        skiprows=kwargs['skiprows']
    )

    # Pluck the columns we want
    trimmed_df = df[kwargs['columns']]

    # Add the year of the source
    trimmed_df['year'] = kwargs['year']
    
    # Clean up header names
    trimmed_df.columns = gen_columns

    # Return the polished dataframe
    return trimmed_df

Create an empty DataFrame

In [22]:
gen_df = pd.DataFrame(columns=gen_columns)

Load all the source files in

In [23]:
for d in gen_file_list:
    print "Standardizing {}".format(d['year'])
    gen_df = gen_df.append(standardize_generation(**d))

Standardizing 2001
Standardizing 2002
Standardizing 2003
Standardizing 2004
Standardizing 2005
Standardizing 2006
Standardizing 2007
Standardizing 2008
Standardizing 2009
Standardizing 2010
Standardizing 2011
Standardizing 2012
Standardizing 2013
Standardizing 2014
Standardizing 2015


Group the data by generator to reduce it to one row per plants

In [25]:
plant_gen_df = gen_df.groupby([
    "year",
    "plant_id",
    'plant_name',
    'operator_name',
    "operator_id"
])['net_generation_mwh'].sum().reset_index()

## Capacity

List the capacity source files and their configuration for pandas

In [64]:
cap_file_list = [
    dict(path='{}01.dbf', skiprows=0, year=2001, sheetname="GENY01", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'PLNTSTATE', 'GENCODE', 'NAMEPLATE', 'SUMMCAP']),
    dict(path='{}02.dbf', skiprows=0, year=2002, sheetname="GENY02", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'PLNTSTATE', 'GENCODE', 'NAMEPLATE', 'SUMMCAP']),
    dict(path='{}03.dbf', skiprows=0, year=2003, sheetname="GENY03", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'PLNTSTATE', 'GENCODE', 'NAMEPLATE', 'SUMMCAP']),
    dict(path='GenY04.xls', skiprows=0, year=2004, sheetname="GenY04", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'STATE', 'GENCODE', 'NAMEPLATE', 'SUMMCAP']),
    dict(path='GenY05.xls', skiprows=0, year=2005, sheetname="GenY05", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'STATE', 'GENCODE', 'NAMEPLATE', 'SUMMCAP']),
    dict(path='GenY06.xls', skiprows=0, year=2006, sheetname="GenY06", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'STATE', 'GENCODE', 'NAMEPLATE', 'SUMMCAP']),
    dict(path='GenY07.xls', skiprows=0, year=2007, sheetname="GenY07", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'STATE', 'GENCODE', 'NAMEPLATE', 'SUMMER_CAPACITY']),
    dict(path='GenY08.xls', skiprows=0, year=2008, sheetname="GenY08", columns=['PLNTCODE', 'PLNTNAME', 'UTILNAME', 'UTILCODE', 'STATE', 'GENCODE', 'NAMEPLATE', 'SUMMER_CAPABILITY']),
    dict(path='GeneratorY09.xls', skiprows=0, year=2009, sheetname="Exist", columns=["PLANT_CODE", "PLANT_NAME", "UTILITY_NAME", "UTILITY_ID", 'STATE', "GENERATOR_ID", "NAMEPLATE", "SUMMER_CAPABILITY"]),
    dict(path='GeneratorsY2010.xls', sheetname='Exist', skiprows=0, year=2010, columns=["PLANT_CODE", "PLANT_NAME", "UTILITY_NAME", "UTILITY_ID", 'STATE', "GENERATOR_ID", "NAMEPLATE", "SUMMER_CAPABILITY"]),
    dict(path='GeneratorY2011.xlsx', sheetname="operable", year=2011, columns=["PLANT_CODE", "PLANT_NAME", "UTILITY_NAME", "UTILITY_ID", 'STATE', "GENERATOR_ID", "NAMEPLATE", "SUMMER_CAPABILITY"]),
    dict(path='GeneratorY2012.xlsx', year=2012, columns=["Plant Code", 'Plant Name', 'Utility Name', 'Utility ID', 'State', 'Generator ID', 'Nameplate Capacity (MW)', 'Summer Capacity (MW)']),
    dict(path='3_1_Generator_Y2013.xlsx', year=2013, columns=["Plant Code", 'Plant Name', 'Utility Name', 'Utility ID', 'State', 'Generator ID', 'Nameplate Capacity (MW)', 'Summer Capacity (MW)']),
    dict(path='3_1_Generator_Y2014.xlsx', year=2014, columns=["Plant Code", 'Plant Name', 'Utility Name', 'Utility ID', 'State', 'Generator ID', 'Nameplate Capacity (MW)', 'Summer Capacity (MW)']),
    dict(path='3_1_Generator_Y2015.xlsx', year=2015, columns=["Plant Code", 'Plant Name', 'Utility Name', 'Utility ID', 'State', 'Generator ID', 'Nameplate Capacity (MW)', 'Summer Capacity (MW)']),
]

Combine files

In [65]:
cap_columns = [
    'plant_id',
    'plant_name',
    'operator_name',
    'operator_id',
    'state',
    'generator_id',
    'nameplate_mw',
    'summer_mw',
    'year'
]

In [66]:
def combine_capacity_dbfs(**kwargs):
    """
    Combine a set of capacity DBF files.
    
    Returns a DataFrame.
    """
    # Open all three DBFs
    info_path = os.path.join(settings.input_dir, kwargs['path'].upper().format("PLANTY"))
    info_df = Dbf5(info_path).to_dataframe()

    stat_path = os.path.join(settings.input_dir, kwargs['path'].format("GENY"))
    stat_df = Dbf5(stat_path).to_dataframe()
    
    owner_path = os.path.join(settings.input_dir, kwargs['path'].upper().format("UTILY"))
    owner_df = Dbf5(owner_path).to_dataframe()
    
    # Merge them together and pass it out                          
    return pd.merge(pd.merge(info_df ,owner_df), stat_df)


In [67]:
def standardize_capacity(**kwargs):
    """
    Standardize a given capacity file to a common set of fields.
    
    Returns a DataFrame.
    """
    # If it's a dbf file, open it using our special trick
    if "dbf" in kwargs['path']:
        df = combine_capacity_dbfs(**kwargs)
    # Otherwise, assume it's Excel
    else:
        df = pd.read_excel(
            os.path.join(settings.input_dir, kwargs['path']),
            sheetname=kwargs.get("sheetname", "Operable"),
            skiprows=kwargs.get("skiprows", 1)
        )
    
    # Pluck the columns we want
    trimmed_df = df[kwargs['columns']]

    # Add the year of the source
    trimmed_df['year'] = kwargs['year']
    
    # Clean up header names
    trimmed_df.columns = cap_columns
    
    # Pass it back
    return trimmed_df

In [68]:
cap_df = pd.DataFrame(columns=cap_columns)

In [69]:
for d in cap_file_list:
    print "Standardizing {}".format(d['year'])
    cap_df = cap_df.append(standardize_capacity(**d))

Standardizing 2001
Standardizing 2002
Standardizing 2003
Standardizing 2004
Standardizing 2005
Standardizing 2006
Standardizing 2007
Standardizing 2008
Standardizing 2009
Standardizing 2010
Standardizing 2011
Standardizing 2012
Standardizing 2013
Standardizing 2014
Standardizing 2015


Filling blanks

In [70]:
cap_df['plant_name'].fillna("", inplace=True)
cap_df['operator_name'].fillna("", inplace=True)
cap_df['state'].fillna("", inplace=True)
cap_df['summer_mw'] = cap_df['summer_mw'].replace(" ", 0.0).replace(".", 0.0).astype(float)

Group to reduce file to one row per plant

In [73]:
plant_cap_df = cap_df.groupby([
    "year",
    "plant_id",
    'plant_name',
    'operator_name',
    "operator_id",
    'state'
])[['nameplate_mw', 'summer_mw',]].sum().reset_index()

## Utilization

Join the two tables

In [94]:
utilization_df = pd.merge(
    plant_gen_df,
    plant_cap_df,
    on=["year", "plant_id"],
    how='outer',
    suffixes=['_gen','_cap']
).sort_values('year')

Convert the capacity value to the same units as the generation

In [95]:
def convert_mw_to_mwh(mw):
    """
    Accepts a Megawatts (MW) value and returns it converted to MegaWatt Hours (MWh)
    """
    return mw * 8760.0

In [96]:
utilization_df['capacity_mwh'] = convert_mw_to_mwh(utilization_df['summer_mw'])

Calculate the percentage of each plant's capacity that was utilized

In [97]:
utilization_df['capacity_utilization'] = utilization_df['net_generation_mwh'] / utilization_df['capacity_mwh']

### Age

Calculate the minimum operating year for any generator at every plant

In [98]:
age_df = pd.read_excel(os.path.join(settings.input_dir, '3_1_Generator_Y2015.xlsx'), skiprows=1)

In [99]:
age_df = age_df.groupby("Plant Code")['Operating Year'].min().reset_index()

Merge that with our utilization data frame

In [100]:
utilization_df = utilization_df.merge(age_df, left_on="plant_id", right_on="Plant Code", how="left")

In [101]:
utilization_df.rename(columns={"Operating Year": "minimum_year"}, inplace=True)

In [102]:
utilization_df.drop("Plant Code", axis=1, inplace=True)

### Export

Write out a CSV

In [107]:
utilization_df.to_csv(os.path.join(settings.output_dir, "plants.csv"), index=False)

Trim it to California

In [108]:
california_utilization_df = utilization_df[utilization_df.state == 'CA']

In [109]:
california_utilization_df.to_csv(os.path.join(settings.output_dir, "plants-california.csv"), index=False)